In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[5]").appName("VarunBatches").getOrCreate()

25/06/19 11:45:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/19 11:46:00 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/19 11:46:00 WARN Utils: Serv

In [7]:
rides = spark.read.csv("sparkstream/rides.csv", inferSchema=True, header=True, sep=',')

In [8]:
rides.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ride_id: string (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- driver_id: string (nullable = true)



In [18]:
rides.show(5)

+--------------------+--------------------+------------+---------+
|             ride_id|tpep_pickup_datetime|PULocationID|driver_id|
+--------------------+--------------------+------------+---------+
|39028a1a-417c-486...| 2025-06-10 08:01:36|           4|     D009|
|4fa830d2-8ea2-48a...| 2025-06-10 10:27:29|          29|     D005|
|2ea3af67-15ee-48e...| 2025-06-10 07:51:57|          87|     D018|
|d45b00db-819a-4dd...| 2025-06-10 07:34:57|          76|     D014|
|8c4ee6d3-efd0-4c3...| 2025-06-10 06:34:42|           4|     D003|
+--------------------+--------------------+------------+---------+
only showing top 5 rows



In [15]:
rides = rides.drop("_c0")

In [21]:
rides.createTempView("rides_data")

In [22]:
ride_count = spark.sql("select driver_id, count(*) as rides_count from rides_data group by driver_id order by rides_count desc")

In [23]:
ride_count.show()

+---------+-----------+
|driver_id|rides_count|
+---------+-----------+
|     D013|        435|
|     D005|        431|
|     D019|        427|
|     D004|        423|
|     D016|        414|
|     D018|        414|
|     D006|        410|
|     D009|        408|
|     D020|        406|
|     D011|        405|
|     D014|        400|
|     D015|        399|
|     D002|        397|
|     D008|        392|
|     D010|        388|
|     D012|        388|
|     D003|        387|
|     D007|        362|
|     D017|        359|
|     D001|        355|
+---------+-----------+



In [35]:
spark.sql("select driver_id,count(*),hour(tpep_pickup_datetime) as hr from rides_data group by driver_id,hr").show()

+---------+--------+---+
|driver_id|count(1)| hr|
+---------+--------+---+
|     D009|      45| 14|
|     D016|      43| 15|
|     D017|      27| 10|
|     D004|      45| 13|
|     D011|      39|  7|
|     D004|      43| 11|
|     D011|      36| 12|
|     D019|      49| 11|
|     D009|      41| 13|
|     D015|      38| 10|
|     D018|      44|  7|
|     D003|      38| 14|
|     D010|      37| 14|
|     D013|      29| 10|
|     D016|      36| 11|
|     D014|      45| 14|
|     D008|      49| 11|
|     D014|      42|  7|
|     D013|      53| 12|
|     D016|      33| 10|
+---------+--------+---+
only showing top 20 rows



In [37]:
spark.sql("select driver_id,count(*),year(tpep_pickup_datetime) as yy from rides_data group by driver_id,yy").show()

+---------+--------+----+
|driver_id|count(1)|  yy|
+---------+--------+----+
|     D020|     406|2025|
|     D008|     392|2025|
|     D012|     388|2025|
|     D002|     397|2025|
|     D015|     399|2025|
|     D009|     408|2025|
|     D007|     362|2025|
|     D001|     355|2025|
|     D006|     410|2025|
|     D018|     414|2025|
|     D004|     423|2025|
|     D011|     405|2025|
|     D017|     359|2025|
|     D016|     414|2025|
|     D005|     431|2025|
|     D014|     400|2025|
|     D003|     387|2025|
|     D013|     435|2025|
|     D010|     388|2025|
|     D019|     427|2025|
+---------+--------+----+



In [3]:
from pyspark.sql.types import *

In [4]:
sch = StructType([\
                  StructField("ride_id", StringType(), True),\
                  StructField("tpep_pickup_datetime", TimestampType(), True),\
                  StructField("PULocationID", IntegerType(), True),\
                  StructField("driver_id", StringType(), True)])

In [5]:
rides_stream = spark.readStream.format("json").schema(sch).load("sparkstream/streaminputs")

In [6]:
rides_stream.printSchema()

root
 |-- ride_id: string (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- driver_id: string (nullable = true)



In [7]:
from pyspark.sql.functions import *

In [8]:
ride_stream_gr = rides_stream.withWatermark("tpep_pickup_datetime","20 minutes").groupBy(window(rides_stream.tpep_pickup_datetime,'10 minutes'), rides_stream.driver_id).count()

In [9]:
ride_stream_gr.isStreaming

True

In [ ]:
ride_stream_gr.writeStream.outputMode("update").format("console").start().awaitTermination()

25/06/19 11:48:39 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-11477bd7-e7a5-4ffe-b3be-e85d0d3cc548. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 13:30...|     D005|    7|
|{2025-06-10 12:50...|     D007|    3|
|{2025-06-10 10:50...|     D014|    6|
|{2025-06-10 07:50...|     D012|    5|
|{2025-06-10 12:00...|     D013|    2|
|{2025-06-10 10:50...|     D016|    1|
|{2025-06-10 14:40...|     D008|    5|
|{2025-06-10 11:30...|     D009|    4|
|{2025-06-10 07:20...|     D008|    5|
|{2025-06-10 13:10...|     D013|    2|
|{2025-06-10 13:40...|     D016|    5|
|{2025-06-10 11:10...|     D012|    2|
|{2025-06-10 09:50...|     D009|    5|
|{2025-06-10 09:00...|     D006|    3|
|{2025-06-10 11:20...|     D007|    3|
|{2025-06-10 12:20...|     D017|    5|
|{2025-06-10 15:30...|     D005|    5|
|{2025-06-10 15:00...|     D002|    1|
|{2025-06-10 14:20...|     D006|    5|
|{2025-06-10 08:40...|     D015|    4|
+-----

-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 11:20...|     D007|    4|
|{2025-06-10 11:50...|     D018|    4|
|{2025-06-10 10:40...|     D017|    2|
|{2025-06-10 13:30...|     D013|    5|
|{2025-06-10 14:30...|     D013|    2|
|{2025-06-10 07:50...|     D008|    6|
|{2025-06-10 11:30...|     D012|    5|
|{2025-06-10 07:40...|     D011|    5|
|{2025-06-10 08:00...|     D006|    7|
|{2025-06-10 14:50...|     D011|   10|
|{2025-06-10 15:10...|     D016|    7|
|{2025-06-10 10:00...|     D016|    1|
|{2025-06-10 11:30...|     D005|    3|
|{2025-06-10 13:40...|     D013|    8|
|{2025-06-10 15:20...|     D010|    1|
|{2025-06-10 10:40...|     D020|    4|
|{2025-06-10 13:10...|     D006|    2|
|{2025-06-10 12:30...|     D007|    6|
|{2025-06-10 15:40...|     D007|    8|
|{2025-06-10 14:30...|     D003|    5|
+-----

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 14:30...|     D007|    3|
|{2025-06-10 07:50...|     D018|    9|
|{2025-06-10 06:10...|     D014|    6|
|{2025-06-10 08:00...|     D006|    8|
|{2025-06-10 08:40...|     D005|    5|
|{2025-06-10 11:00...|     D015|    3|
|{2025-06-10 14:50...|     D017|    5|
|{2025-06-10 15:20...|     D019|    3|
|{2025-06-10 13:00...|     D001|    2|
|{2025-06-10 07:20...|     D003|    4|
|{2025-06-10 12:00...|     D003|    6|
|{2025-06-10 11:20...|     D020|    3|
|{2025-06-10 09:40...|     D010|    3|
|{2025-06-10 07:00...|     D012|    6|
|{2025-06-10 11:50...|     D008|    4|
|{2025-06-10 09:40...|     D006|    6|
|{2025-06-10 08:40...|     D011|    7|
|{2025-06-10 06:30...|     D011|    7|
|{2025-06-10 07:10...|     D014|    8|
|{2025-06-10 08:40...|     D008|    6|
+-----

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 12:50...|     D015|    8|
|{2025-06-10 10:10...|     D012|    4|
|{2025-06-10 09:40...|     D013|    6|
|{2025-06-10 13:10...|     D012|    4|
|{2025-06-10 12:30...|     D011|    4|
|{2025-06-10 08:10...|     D001|    3|
|{2025-06-10 08:10...|     D005|    9|
|{2025-06-10 14:40...|     D017|    7|
|{2025-06-10 08:10...|     D007|    5|
|{2025-06-10 13:00...|     D001|    3|
|{2025-06-10 08:30...|     D016|    9|
|{2025-06-10 07:30...|     D012|    5|
|{2025-06-10 13:50...|     D010|    6|
|{2025-06-10 09:40...|     D010|    4|
|{2025-06-10 13:30...|     D007|    6|
|{2025-06-10 06:00...|     D014|    4|
|{2025-06-10 12:30...|     D003|    8|
|{2025-06-10 08:50...|     D009|    4|
|{2025-06-10 11:00...|     D011|    3|
|{2025-06-10 08:00...|     D001|    3|
+-----

...

-------------------------------------------
Batch: 7
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 10:50...|     D016|    2|
|{2025-06-10 09:20...|     D013|    2|
|{2025-06-10 14:00...|     D011|    7|
|{2025-06-10 06:30...|     D015|    3|
|{2025-06-10 09:00...|     D001|    6|
|{2025-06-10 13:10...|     D012|    5|
|{2025-06-10 13:40...|     D002|    4|
|{2025-06-10 12:30...|     D020|    2|
|{2025-06-10 08:50...|     D018|    8|
|{2025-06-10 11:50...|     D009|    3|
|{2025-06-10 15:40...|     D008|    8|
|{2025-06-10 11:40...|     D016|    3|
|{2025-06-10 09:10...|     D011|    4|
|{2025-06-10 12:50...|     D017|    5|
|{2025-06-10 11:00...|     D017|    6|
|{2025-06-10 15:50...|     D011|   10|
|{2025-06-10 11:40...|     D004|    7|
|{2025-06-10 09:00...|     D003|    7|
|{2025-06-10 06:40...|     D014|    3|
|{2025-06-10 07:40...|     D014|    6|
+-----

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 07:20...|     D008|    6|
|{2025-06-10 13:10...|     D013|    4|
|{2025-06-10 09:20...|     D006|    4|
|{2025-06-10 10:00...|     D012|    5|
|{2025-06-10 10:20...|     D020|    9|
|{2025-06-10 13:20...|     D005|    4|
|{2025-06-10 14:00...|     D008|    4|
|{2025-06-10 07:20...|     D003|    5|
|{2025-06-10 09:50...|     D008|    6|
|{2025-06-10 11:50...|     D008|    5|
|{2025-06-10 07:20...|     D005|    6|
|{2025-06-10 12:40...|     D008|    2|
|{2025-06-10 15:40...|     D013|   10|
|{2025-06-10 13:00...|     D009|    4|
|{2025-06-10 12:30...|     D014|    5|
|{2025-06-10 15:40...|     D004|    7|
|{2025-06-10 15:50...|     D009|    2|
|{2025-06-10 07:40...|     D016|    3|
|{2025-06-10 06:20...|     D008|    4|
|{2025-06-10 15:20...|     D013|    9|
+-----

-------------------------------------------
Batch: 9
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 09:20...|     D013|    3|
|{2025-06-10 07:40...|     D019|    8|
|{2025-06-10 11:30...|     D010|    2|
|{2025-06-10 10:30...|     D006|    3|
|{2025-06-10 07:00...|     D016|    2|
|{2025-06-10 08:40...|     D012|    5|
|{2025-06-10 12:00...|     D019|    7|
|{2025-06-10 10:30...|     D001|    2|
|{2025-06-10 08:10...|     D003|    6|
|{2025-06-10 13:10...|     D015|    4|
|{2025-06-10 07:00...|     D014|    2|
|{2025-06-10 15:10...|     D005|    7|
|{2025-06-10 13:50...|     D006|    4|
|{2025-06-10 10:30...|     D003|    6|
|{2025-06-10 14:30...|     D011|    7|
|{2025-06-10 13:50...|     D019|    6|
|{2025-06-10 06:50...|     D012|    3|
|{2025-06-10 10:00...|     D009|    6|
|{2025-06-10 15:40...|     D019|    3|
|{2025-06-10 09:10...|     D016|    5|
+-----

-------------------------------------------
Batch: 10
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 11:10...|     D012|    3|
|{2025-06-10 14:40...|     D019|    7|
|{2025-06-10 07:10...|     D008|    6|
|{2025-06-10 09:00...|     D016|    7|
|{2025-06-10 08:40...|     D012|    6|
|{2025-06-10 09:30...|     D005|    6|
|{2025-06-10 12:30...|     D008|    8|
|{2025-06-10 15:00...|     D018|    9|
|{2025-06-10 07:20...|     D015|    9|
|{2025-06-10 15:30...|     D001|    5|
|{2025-06-10 07:40...|     D001|    5|
|{2025-06-10 09:10...|     D006|    4|
|{2025-06-10 09:50...|     D004|    6|
|{2025-06-10 07:00...|     D012|    7|
|{2025-06-10 15:10...|     D015|    4|
|{2025-06-10 11:50...|     D010|    8|
|{2025-06-10 06:30...|     D019|    5|
|{2025-06-10 06:30...|     D002|    6|
|{2025-06-10 09:10...|     D017|    3|
|{2025-06-10 13:00...|     D004|    7|
+----

-------------------------------------------
Batch: 11
-------------------------------------------
+--------------------+---------+-----+
|              window|driver_id|count|
+--------------------+---------+-----+
|{2025-06-10 14:50...|     D011|   11|
|{2025-06-10 11:00...|     D005|    7|
|{2025-06-10 13:20...|     D013|    2|
|{2025-06-10 11:10...|     D014|    2|
|{2025-06-10 07:20...|     D001|    4|
|{2025-06-10 15:40...|     D013|   11|
|{2025-06-10 14:40...|     D013|    9|
|{2025-06-10 15:50...|     D007|    4|
|{2025-06-10 11:50...|     D019|    7|
|{2025-06-10 14:00...|     D007|    8|
+--------------------+---------+-----+

